# **I차. Feature Engineering**

라이브러리

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

데이터 불러오기

In [ ]:
df_enterprise_sample = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/기업데이터 형태샘플(AIB전달용) - 학습샘플.csv')

In [ ]:
df_enterprise_sample.head()

In [ ]:
df_enterprise_sample.columns

Index(['섹터', '기업체명', '영문기업명', '사업자번호', '법인(주민)번호', '대표자명', '종업원수', '설립형태',
       '기업체명.1', '설립일자', '기업형태', '전화번호', '팩스번호', '홈페이지', '이메일', '결산월',
       '기업공개일자', '도로명', '지번', '업종(10차)', '업종(9차)', '주요제품(상품)', '무역업허가번호',
       '소속그룹', '주채권기관', '당좌거래은행', '휴폐업정보', '법인등기정보'],
      dtype='object')

In [ ]:
df_enterprise_sample_tag = df_enterprise_sample

In [ ]:
df_enterprise_sample_tag = df_enterprise_sample_tag.rename(columns={'기업체명.1':'설립일자', '설립일자':'기업형태','기업형태':'기업규모','소속그룹':'계열사여부'})

In [ ]:
df_enterprise_sample_tag = df_enterprise_sample_tag.drop(['영문기업명','사업자번호','법인(주민)번호','대표자명','전화번호','팩스번호','홈페이지','이메일','결산월','기업공개일자','도로명','업종(9차)','주채권기관',
                                                          '당좌거래은행','법인등기정보'], axis=1)

In [ ]:
df_enterprise_sample_tag.columns

Index(['섹터', '기업체명', '종업원수', '설립형태', '설립일자', '기업형태', '기업규모', '지번', '업종(10차)',
       '주요제품(상품)', '무역업허가번호', '계열사여부', '휴폐업정보'],
      dtype='object')

## 키워드 추출 ##

### 종업원수 ###

In [ ]:
# 종업원수 조정
def worker_int(s):
    if s.startswith('명') == True:
        s = s.replace('명', '0명')
    if '명' in s:
        s = s.replace('명','')
        if ',' in s:
            s = s.replace(',','')
    return int(s)

In [ ]:
# 종업원수 조정 함수 적용
df_enterprise_sample_tag['종업원수'] = df_enterprise_sample_tag['종업원수'].apply(worker_int)
df_enterprise_sample_tag.head()

,섹터,기업체명,종업원수,설립형태,설립일자,기업형태,기업규모,지번,업종(10차),주요제품(상품),무역업허가번호,계열사여부,휴폐업정보
0,공공행정국방및사회보장행정,금융감독원,1589,신규설립(개업),1999-01-02,일반법인,판단제외,(07321) 서울 영등포구 여의도동 27번지 금융감독원,(K66110)금융시장 관리업,금융기관에 대한 검사.감독 업무 등,-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-05)
1,공공행정국방및사회보장행정,(주)명현유통,17,신규설립(개업),2001-04-26,외감,중기업,(17396) 경기 이천시 호법면 매곡리 990-8번지,(G46332)비알코올음료 도매업,"음료, 잡화, 식자재, 냉동식품 등",-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-05)
2,공공행정국방및사회보장행정,서울관광마케팅(주),67,신규설립(개업),2008-02-04,일반법인,중기업,(04551) 서울 중구 저동1가 1-2번지 나라키움저동빌딩 9층,(O84212)문화 및 관광 행정,문화 및 관광 행정,-,-,폐업자\n(폐업일자 : 2019-07-31)
3,공공행정국방및사회보장행정,(주)씨티엘네트웍스,65,신규설립(개업),2006-05-17,일반법인,중기업,(06104) 서울 강남구 논현2동 210-1번지 삼원빌딩 8층,(N75210)여행사업,관광서비스 외,-,-,폐업자\n(폐업일자 : 2015-06-30)
4,공공행정국방및사회보장행정,(주)용준,1,신규설립(개업),2019-04-17,일반법인,소상공인,(61949) 광주 서구 치평동 1217-10번지 종산타워 4층 401호 일부,(G47121)슈퍼마켓,"슈퍼마켓(와이마트), 건설장비운영, 부동산임대",-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-04)


In [ ]:
df_enterprise_sample_tag = df_enterprise_sample_tag[['섹터', '기업체명', '종업원수', '설립형태', '설립일자', '기업형태', '기업규모', '지번', '업종(10차)',
       '주요제품(상품)', '무역업허가번호', '소속그룹', '휴폐업정보']]

### 업력###

#### 업력(연) ####

In [ ]:
# 업력(연)
def enterprise_age_year(string):
    if string == '-':
        pass
    else:
        today = datetime.today()
        y = today.year
        m = today.month
        d = today.day
        if int(string[5:7]) < m:
            string = y - int(string[:4])
        elif int(string[5:7]) == m:
            if int(string[8:10]) <= d:
                string = y - int(string[:4])
            elif int(string[8:10]) > d:
                string = y - int(string[:4]) - 1
        elif int(string[5:7]) > m:
            string = y - int(string[:4]) - 1
    return string

In [ ]:
# 업력(연)
df_enterprise_sample_tag['업력(연)'] = df_enterprise_sample_tag['설립일자']
df_enterprise_sample_tag['업력(연)'] = df_enterprise_sample_tag['업력(연)'].apply(enterprise_age_year)

In [ ]:
df_enterprise_sample_tag.head()

,섹터,기업체명,종업원수,설립형태,설립일자,기업형태,기업규모,지번,업종(10차),주요제품(상품),무역업허가번호,계열사여부,휴폐업정보,업력(연)
0,공공행정국방및사회보장행정,금융감독원,1589,신규설립(개업),1999-01-02,일반법인,판단제외,(07321) 서울 영등포구 여의도동 27번지 금융감독원,(K66110)금융시장 관리업,금융기관에 대한 검사.감독 업무 등,-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-05),23
1,공공행정국방및사회보장행정,(주)명현유통,17,신규설립(개업),2001-04-26,외감,중기업,(17396) 경기 이천시 호법면 매곡리 990-8번지,(G46332)비알코올음료 도매업,"음료, 잡화, 식자재, 냉동식품 등",-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-05),20
2,공공행정국방및사회보장행정,서울관광마케팅(주),67,신규설립(개업),2008-02-04,일반법인,중기업,(04551) 서울 중구 저동1가 1-2번지 나라키움저동빌딩 9층,(O84212)문화 및 관광 행정,문화 및 관광 행정,-,-,폐업자\n(폐업일자 : 2019-07-31),13
3,공공행정국방및사회보장행정,(주)씨티엘네트웍스,65,신규설립(개업),2006-05-17,일반법인,중기업,(06104) 서울 강남구 논현2동 210-1번지 삼원빌딩 8층,(N75210)여행사업,관광서비스 외,-,-,폐업자\n(폐업일자 : 2015-06-30),15
4,공공행정국방및사회보장행정,(주)용준,1,신규설립(개업),2019-04-17,일반법인,소상공인,(61949) 광주 서구 치평동 1217-10번지 종산타워 4층 401호 일부,(G47121)슈퍼마켓,"슈퍼마켓(와이마트), 건설장비운영, 부동산임대",-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-04),2


#### 업력(월) ####

In [ ]:
# 업력(월)
def enterprise_age_month(string):
    if string == '-':
        pass
    else:
        today = datetime.today()
        y = today.year
        m = today.month
        d = today.day
        if int(string[5:7]) < m:
            string = (y - int(string[:4])) * 12 + int(string[5:7])
        elif int(string[5:7]) == m:
            if int(string[8:10]) <= d:
                string = (y - int(string[:4])) * 12 + int(string[5:7])
            elif int(string[8:10]) > d:
                string = (y - int(string[:4]) -1) * 12 + int(string[5:7])
        elif int(string[5:7]) > m:
            string = (y - int(string[:4]) -1) * 12 + int(string[5:7])
    return string

In [ ]:
# 업력(연)
df_enterprise_sample_tag['업력(월)'] = df_enterprise_sample_tag['설립일자']
df_enterprise_sample_tag['업력(월)'] = df_enterprise_sample_tag['업력(월)'].apply(enterprise_age_month)

### 무역업 허가여부 ###

In [ ]:
# 무역업 허가번호
def trade_permission(string):
    if string == '-':
        string = '해당없음'
    else:
        string = '해당'
    return string

In [ ]:
df_enterprise_sample_tag['무역업허가여부'] = df_enterprise_sample_tag['무역업허가번호'].apply(trade_permission)

In [ ]:
df_enterprise_sample_tag.head()

,섹터,기업체명,종업원수,설립형태,설립일자,기업형태,기업규모,지번,업종(10차),주요제품(상품),무역업허가번호,계열사여부,휴폐업정보,업력(연),업력(월),무역업허가여부
0,공공행정국방및사회보장행정,금융감독원,1589,신규설립(개업),1999-01-02,일반법인,판단제외,(07321) 서울 영등포구 여의도동 27번지 금융감독원,(K66110)금융시장 관리업,금융기관에 대한 검사.감독 업무 등,-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-05),23,277,해당없음
1,공공행정국방및사회보장행정,(주)명현유통,17,신규설립(개업),2001-04-26,외감,중기업,(17396) 경기 이천시 호법면 매곡리 990-8번지,(G46332)비알코올음료 도매업,"음료, 잡화, 식자재, 냉동식품 등",-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-05),20,244,해당없음
2,공공행정국방및사회보장행정,서울관광마케팅(주),67,신규설립(개업),2008-02-04,일반법인,중기업,(04551) 서울 중구 저동1가 1-2번지 나라키움저동빌딩 9층,(O84212)문화 및 관광 행정,문화 및 관광 행정,-,-,폐업자\n(폐업일자 : 2019-07-31),13,158,해당없음
3,공공행정국방및사회보장행정,(주)씨티엘네트웍스,65,신규설립(개업),2006-05-17,일반법인,중기업,(06104) 서울 강남구 논현2동 210-1번지 삼원빌딩 8층,(N75210)여행사업,관광서비스 외,-,-,폐업자\n(폐업일자 : 2015-06-30),15,185,해당없음
4,공공행정국방및사회보장행정,(주)용준,1,신규설립(개업),2019-04-17,일반법인,소상공인,(61949) 광주 서구 치평동 1217-10번지 종산타워 4층 401호 일부,(G47121)슈퍼마켓,"슈퍼마켓(와이마트), 건설장비운영, 부동산임대",-,-,부가가치세 일반과세자\n(조회일자 : 2022-01-04),2,28,해당없음


### 계열사 여부 ###

In [ ]:
def family_group(string):
    if string == '-':
        string = '해당없음'
    else:
        string = '계열사'
    return string

In [ ]:
df_enterprise_sample_tag['계열사여부'] = df_enterprise_sample_tag['계열사여부'].apply(family_group)

### 휴폐업 여부, 휴폐업일 ###

In [ ]:
# 휴폐업
def enterprise_close(string):
    if string.startswith('폐업'):
        string = '폐업'
    elif string.startswith('휴업'):
        string = '휴업'
    else:
        string = '해당없음'
    return string

In [ ]:
df_enterprise_sample_tag['휴폐업일'] = df_enterprise_sample_tag['휴폐업정보']

In [ ]:
df_enterprise_sample_tag['휴폐업정보'] = df_enterprise_sample_tag['휴폐업정보'].apply(enterprise_close)

In [ ]:
for i in range(len(df_enterprise_sample_tag)):
    if df_enterprise_sample_tag['휴폐업정보'][i].startswith('폐업') == True:
        pass
    elif df_enterprise_sample_tag['휴폐업정보'][i].startswith('휴업') == True:
        pass
    else:
        df_enterprise_sample_tag['휴폐업일'][i] = '해당없음'

In [ ]:
text = '폐업자\n(폐업일자 : 2019-07-31)'
text[12:22]

'2019-07-31'

In [ ]:
# 휴폐업일
def enterprise_close_date(string):
    if string == '해당없음':
        pass
    else:
        string = string
    return string[12:22]

In [ ]:
df_enterprise_sample_tag['휴폐업일'] = df_enterprise_sample_tag['휴폐업일'].apply(enterprise_close_date)

### 지역(광역시도, 시군구) 추출 ###

In [ ]:
df_enterprise_sample_tag['지역(광역시도)'] = 0
df_enterprise_sample_tag['지역(시군구)'] = 0

In [ ]:
text = '(13616) 경기 성남시 분당구 금곡동 210번지'
text.split(' ')[2]

'성남시'

In [ ]:
for i in range(len(df_enterprise_sample_tag)):
    s = df_enterprise_sample_tag['지번'][i].split(' ')
    df_enterprise_sample_tag['지역(광역시도)'][i] = s[1]
    df_enterprise_sample_tag['지역(시군구)'][i] = s[2]

In [ ]:
for i in range(len(df_enterprise_sample_tag)):
    if df_enterprise_sample_tag['지역(광역시도)'][i] == '서울':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '서울특별시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '경기':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '경기도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '인천':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '인천광역시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '충북':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '충청북도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '대전':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '대전광역시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '충남':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '충청남도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '세종':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '세종특별자치시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '전북':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '전라북도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '광주':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '광주광역시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '전남':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '전라남도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '제주':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '제주특별자치도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '경남':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '경상남도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '부산':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '부산광역시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '대구':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '대구광역시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '울산':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '울산광역시'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '경북':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '경상북도'
    elif df_enterprise_sample_tag['지역(광역시도)'][i] == '강원':
        df_enterprise_sample_tag['지역(광역시도)'][i] = '강원도'

### 업종 분류 ###

In [ ]:
df_enterprise_sample_tag['업종(대분류)'] = 0

In [ ]:
# 업종(대분류)
# 한국 표준산업분류(10차)를 참조하여 알파벳 코드에 따라 업종 구분
for i in range(len(df_enterprise_sample_tag)):
    if df_enterprise_sample_tag['업종(10차)'][i].startswith('(A'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '농업, 임업 및 어업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(B'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '광업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(C'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '제조업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(D'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '전기, 가스, 증기 및 공기 조절 공급업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(E'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '수도, 하수 및 폐기물 처리, 원료 재생업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(F'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '건설업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(G'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '도매 및 소매업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(H'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '운수 및 창고업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(I'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '숙박 및 음식점업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(J'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '정보통신업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(K'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '금융 및 보험업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(L'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '부동산업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(M'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '전문, 과학 및 기술 서비스업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(N'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '사업시설 관리, 사업 지원 및 임대 서비스업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(O'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '공공 행정, 국방 및 사회보장 행정'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(P'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '교육 서비스업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(Q'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '보건업 및 사회복지 서비스업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(R'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '예술, 스포츠 및 여가관련 서비스업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(S'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '협회 및 단체, 수리 및 기타 개인 서비스업'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(T'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동'
    elif df_enterprise_sample_tag['업종(10차)'][i].startswith('(U'):
        df_enterprise_sample_tag['업종(대분류)'][i] = '국제 및 외국기관'

In [ ]:
df_enterprise_sample_tag = df_enterprise_sample_tag[['기업체명','종업원수','설립형태','설립일자','업력(연)','업력(월)'
,'기업형태','기업규모','업종(대분류)','주요제품(상품)','무역업허가여부','무역업허가번호','계열사여부','휴폐업정보','휴폐업일','지역(광역시도)','지역(시군구)']]

In [ ]:
df_enterprise_sample_tag.head()

,기업체명,종업원수,설립형태,설립일자,업력(연),업력(월),기업형태,기업규모,업종(대분류),주요제품(상품),무역업허가여부,무역업허가번호,계열사여부,휴폐업정보,휴폐업일,지역(광역시도),지역(시군구)
0,금융감독원,1589,신규설립(개업),1999-01-02,23,277,일반법인,판단제외,금융 및 보험업,금융기관에 대한 검사.감독 업무 등,해당없음,-,해당없음,해당없음,,서울특별시,영등포구
1,(주)명현유통,17,신규설립(개업),2001-04-26,20,244,외감,중기업,도매 및 소매업,"음료, 잡화, 식자재, 냉동식품 등",해당없음,-,해당없음,해당없음,,경기도,이천시
2,서울관광마케팅(주),67,신규설립(개업),2008-02-04,13,158,일반법인,중기업,"공공 행정, 국방 및 사회보장 행정",문화 및 관광 행정,해당없음,-,해당없음,폐업,2019-07-31,서울특별시,중구
3,(주)씨티엘네트웍스,65,신규설립(개업),2006-05-17,15,185,일반법인,중기업,"사업시설 관리, 사업 지원 및 임대 서비스업",관광서비스 외,해당없음,-,해당없음,폐업,2015-06-30,서울특별시,강남구
4,(주)용준,1,신규설립(개업),2019-04-17,2,28,일반법인,소상공인,도매 및 소매업,"슈퍼마켓(와이마트), 건설장비운영, 부동산임대",해당없음,-,해당없음,해당없음,,광주광역시,서구


In [ ]:
index1 = df_enterprise_sample_tag[df_enterprise_sample_tag['휴폐업정보'] == '휴업'].index
index2 = df_enterprise_sample_tag[df_enterprise_sample_tag['휴폐업정보'] == '폐업'].index

In [ ]:
len(index1)

2

In [ ]:
df_enterprise_sample_tag = df_enterprise_sample_tag.drop(index1)
df_enterprise_sample_tag = df_enterprise_sample_tag.drop(index2)

In [ ]:
df_enterprise_sample_tag = df_enterprise_sample_tag[['기업체명','업력(월)','기업형태','기업규모','업종(대분류)','지역(광역시도)','지역(시군구)']]

In [ ]:
df_enterprise_sample_tag.shape

(352, 7)

In [ ]:
df_enterprise_sample_tag.to_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/enterprise_data_ver01.csv', encoding='utf-8-sig',index=False)

In [ ]:
df_enterprise = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/df_enterprise_sample_tag.csv')
df_enterprise

,기업체명,종업원수,설립형태,설립일자,업력(연),업력(월),기업형태,기업규모,업종(대분류),주요제품(상품),무역업허가여부,무역업허가번호,계열사여부,휴폐업여부,휴폐업일,지역(광역시도),지역(시군구)
0,(주)루더비즈코리아,5,신규설립(개업),2002-01-04,20,241,일반법인,소기업,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동,"개인업무대행서비스,기타",해당없음,-,해당없음,폐업,2006-01-06,경기도,성남시
1,금융감독원,1589,신규설립(개업),1999-01-02,23,277,일반법인,판단제외,금융 및 보험업,금융기관에 대한 검사.감독 업무 등,해당없음,-,해당없음,해당없음,NaN,서울특별시,영등포구
2,(주)명현유통,17,신규설립(개업),2001-04-26,20,244,외감,중기업,도매 및 소매업,"음료, 잡화, 식자재, 냉동식품 등",해당없음,-,해당없음,해당없음,NaN,경기도,이천시
3,서울관광마케팅(주),67,신규설립(개업),2008-02-04,13,158,일반법인,중기업,"공공 행정, 국방 및 사회보장 행정",문화 및 관광 행정,해당없음,-,해당없음,폐업,2019-07-31,서울특별시,중구
4,(주)씨티엘네트웍스,65,신규설립(개업),2006-05-17,15,185,일반법인,중기업,"사업시설 관리, 사업 지원 및 임대 서비스업",관광서비스 외,해당없음,-,해당없음,폐업,2015-06-30,서울특별시,강남구
5,(주)용준,1,신규설립(개업),2019-04-17,2,28,일반법인,소상공인,도매 및 소매업,"슈퍼마켓(와이마트), 건설장비운영, 부동산임대",해당없음,-,해당없음,해당없음,NaN,광주광역시,서구
6,웰체인쉽핑(주),62,신규설립(개업),2006-04-17,15,184,일반법인,중기업,운수 및 창고업,"선박관리,선원관리,선박안전관리",해당없음,-,해당없음,해당없음,NaN,울산광역시,남구
7,(재단)전북연구원,74,신규설립(개업),2005-03-10,16,195,일반법인,판단제외,"전문, 과학 및 기술 서비스업시",학술 연구 용역,해당없음,-,해당없음,해당없음,NaN,전라북도,전주시
8,(주)주목,0,신규설립(개업),2019-04-04,2,28,일반법인,-,도매 및 소매업,-,해당없음,-,해당없음,해당없음,NaN,강원도,원주시
9,한국관광공사,759,신규설립(개업),1962-06-26,59,714,일반법인,판단제외,"공공 행정, 국방 및 사회보장 행정","관광진흥사업, 관광자원개발, 관광산업의 연구 및 개발, 관광요원의 양성ㆍ훈련 및 재...",해당,859055,해당없음,해당없음,NaN,강원도,원주시


## 전처리 ##

## 함수 ##

In [ ]:
# 종업원수 조정
def worker_int(s):
    if s.startswith('명') == True:
        s = s.replace('명', '0명')
    if '명' in s:
        s = s.replace('명','')
        if ',' in s:
            s = s.replace(',','')
    return int(s)

In [ ]:
# 업력(연)
def enterprise_age_year(string):
    if string == '-':
        pass
    else:
        today = datetime.today()
        y = today.year
        m = today.month
        d = today.day
        if int(string[5:7]) < m:
            string = y - int(string[:4])
        elif int(string[5:7]) == m:
            if int(string[8:10]) <= d:
                string = y - int(string[:4])
            elif int(string[8:10]) > d:
                string = y - int(string[:4]) - 1
        elif int(string[5:7]) > m:
            string = y - int(string[:4]) - 1
    return string

In [ ]:
# 업력(월)
def enterprise_age_month(string):
    if string == '-':
        pass
    else:
        today = datetime.today()
        y = today.year
        m = today.month
        d = today.day
        if int(string[5:7]) < m:
            string = (y - int(string[:4])) * 12 + int(string[5:7])
        elif int(string[5:7]) == m:
            if int(string[8:10]) <= d:
                string = (y - int(string[:4])) * 12 + int(string[5:7])
            elif int(string[8:10]) > d:
                string = (y - int(string[:4]) -1) * 12 + int(string[5:7])
        elif int(string[5:7]) > m:
            string = (y - int(string[:4]) -1) * 12 + int(string[5:7])
    return string

In [ ]:
# 무역업 허가번호
def trade_permission(string):
    if string == '-':
        string = '해당없음'
    else:
        string = '해당'
    return string

In [ ]:
# 계열사 여부
def family_group(string):
    if string == '-':
        string = '해당없음'
    else:
        string = '계열사'
    return string

In [ ]:
# 휴폐업
def enterprise_close(string):
    if string.startswith('폐업'):
        string = '폐업'
    elif string.startswith('휴업'):
        string = '휴업'
    else:
        string = '해당없음'
    return string

In [ ]:
# 휴폐업일
def enterprise_close_date(string):
    if string == '해당없음':
        pass
    else:
        string = string
    return string[12:22]

# **II차. Feature Engineering**

## 데이터 불러오기 ##

In [ ]:
d_ent = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/기업규모&노폐업.csv')

In [ ]:
d_ent.columns

Index(['사업자등록번호', '기업체명', '기업기본주소', '종업원수', '기업설립일자', '금융감독원법인고유번호', '업종',
       '법인등록번호', '기업규모'],
      dtype='object')

In [ ]:
d_ent_preprocessing = d_ent#[['금융감독원법인고유번호','기업체명','기업설립일자','업종','법인등록번호','기업기본주소','기업상세주소']]

In [ ]:
# d_ent_preprocessing.to_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/d_ent_preprocessing_ver_01.csv', encoding='utf-8-sig',index=False)

In [ ]:
d_ent_preprocessing.head(1)

,사업자등록번호,기업체명,기업기본주소,종업원수,기업설립일자,금융감독원법인고유번호,업종,법인등록번호,기업규모
0,3.128135e+09,(주)다코,충청남도 천안시 청당동 419-12,0,19970611.0,434003,제조업,1.615110e+12,중소기업


### 업종 분류 ###

In [ ]:
# d_ent_preprocessing['업종'] = d_ent_preprocessing['업종'].fillna('-')

In [ ]:
# 업종 분류
"""
for i in range(len(d_ent_preprocessing)):
    if d_ent_preprocessing['업종'][i] == '-':
        pass
    elif d_ent_preprocessing['업종'][i] == 'F':
        d_ent_preprocessing.loc[i, '업종'] = '건설업'
    elif d_ent_preprocessing['업종'][i] == 'L':
        d_ent_preprocessing.loc[i, '업종'] = '부동산업'
    elif d_ent_preprocessing['업종'][i] == '1':
        d_ent_preprocessing.loc[i, '업종'] = '농업, 임업 및 어업'
    elif (d_ent_preprocessing['업종'][i] == '5')|(d_ent_preprocessing['업종'][i] == '7')|(d_ent_preprocessing['업종'][i] == '8000'):
        d_ent_preprocessing.loc[i, '업종'] = '광업'
    else:
        if (d_ent_preprocessing['업종'][i][:2] == '01')|(d_ent_preprocessing['업종'][i][:2] == '02')|(d_ent_preprocessing['업종'][i][:2] == '03')|(d_ent_preprocessing['업종'][i] == '1'):
            d_ent_preprocessing.loc[i, '업종'] = '농업, 임업 및 어업' # A
        elif (d_ent_preprocessing['업종'][i][:2] == '05')|(d_ent_preprocessing['업종'][i][:2] == '06')|(d_ent_preprocessing['업종'][i][:2] == '07')|(d_ent_preprocessing['업종'][i][:2] == '08'):
            d_ent_preprocessing.loc[i, '업종'] = '광업' # B
        elif (10 <= int(d_ent_preprocessing['업종'][i][:2]) <= 34):
            d_ent_preprocessing.loc[i, '업종'] = '제조업' # C
        elif int(d_ent_preprocessing['업종'][i][:2]) == 35:
            d_ent_preprocessing.loc[i, '업종'] = '전기, 가스, 증기 및 공기 조절 공급업' # D
        elif 36 <= int(d_ent_preprocessing['업종'][i][:2]) <= 39:
            d_ent_preprocessing.loc[i, '업종'] = '수도, 하수 및 폐기물 처리, 원료 재생업' # E
        elif 41 <= int(d_ent_preprocessing['업종'][i][:2]) <= 42:
            d_ent_preprocessing.loc[i, '업종'] = '건설업' # F
        elif 45 <= int(d_ent_preprocessing['업종'][i][:2]) <= 47:
            d_ent_preprocessing.loc[i, '업종'] = '도매 및 소매업' # G
        elif 49 <= int(d_ent_preprocessing['업종'][i][:2]) <= 52:
            d_ent_preprocessing.loc[i, '업종'] = '운수 및 창고업' # H
        elif 55 <= int(d_ent_preprocessing['업종'][i][:2]) <= 56:
            d_ent_preprocessing.loc[i, '업종'] = '제조업' # I
        elif 58 <= int(d_ent_preprocessing['업종'][i][:2]) <= 63:
            d_ent_preprocessing.loc[i, '업종'] = '정보통신업' # J
        elif 64 <= int(d_ent_preprocessing['업종'][i][:2]) <= 66:
            d_ent_preprocessing.loc[i, '업종'] = '금융 및 보험업' # K
        elif (int(d_ent_preprocessing['업종'][i][:2]) == 68)|(d_ent_preprocessing['업종'][i].startswith('L')):
            d_ent_preprocessing.loc[i, '업종'] = '부동산업' # L
        elif 70 <= int(d_ent_preprocessing['업종'][i][:2]) <= 73:
            d_ent_preprocessing.loc[i, '업종'] = '전문, 과학 및 기술 서비스업' # M
        elif 74 <= int(d_ent_preprocessing['업종'][i][:2]) <= 76:
            d_ent_preprocessing.loc[i, '업종'] = '사업시설 관리, 사업 지원 및 임대 서비스업' # N
        elif int(d_ent_preprocessing['업종'][i][:2]) == 84:
            d_ent_preprocessing.loc[i, '업종'] = '공공 행정, 국방 및 사회보장 행정' # O
        elif int(d_ent_preprocessing['업종'][i][:2]) == 85:
            d_ent_preprocessing.loc[i, '업종'] = '교육 서비스업' # P
        elif 86 <= int(d_ent_preprocessing['업종'][i][:2]) <= 87:
            d_ent_preprocessing.loc[i, '업종'] = '보건업 및 사회복지 서비스업' # Q
        elif 90 <= int(d_ent_preprocessing['업종'][i][:2]) <= 91:
            d_ent_preprocessing.loc[i, '업종'] = '예술, 스포츠 및 여가관련 서비스업' # R
        elif 94 <= int(d_ent_preprocessing['업종'][i][:2]) <= 96:
            d_ent_preprocessing.loc[i, '업종'] = '협회 및 단체, 수리 및 기타 개인 서비스업' # S
        elif 97 <= int(d_ent_preprocessing['업종'][i][:2]) <= 98:
            d_ent_preprocessing.loc[i, '업종'] = '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동' # T
        elif int(d_ent_preprocessing['업종'][i][:2]) == 99:
            d_ent_preprocessing.loc[i, '업종'] = '국제 및 외국기관' # U
"""

In [ ]:
# d_ent_preprocessing['업종'].unique()

array(['제조업', '금융 및 보험업', '건설업', '전문, 과학 및 기술 서비스업', '정보통신업', '부동산업',
       '도매 및 소매업', '운수 및 창고업', '사업시설 관리, 사업 지원 및 임대 서비스업', '교육 서비스업', '-',
       '수도, 하수 및 폐기물 처리, 원료 재생업', '예술, 스포츠 및 여가관련 서비스업',
       '협회 및 단체, 수리 및 기타 개인 서비스업', '전기, 가스, 증기 및 공기 조절 공급업', '광업',
       '보건업 및 사회복지 서비스업', '공공 행정, 국방 및 사회보장 행정', '농업, 임업 및 어업',
       '국제 및 외국기관'], dtype=object)

### 업력 계산 ###

In [ ]:
# 업력 계산
d_ent_preprocessing['기업설립일자'] = d_ent_preprocessing['기업설립일자'].astype(str)

In [ ]:
d_ent_preprocessing['기업설립일자'].isna().sum()

13

In [ ]:
d_ent_preprocessing['기업설립일자'] = d_ent_preprocessing['기업설립일자'].fillna('0')

In [ ]:
d_ent_preprocessing['기업설립일자'].isna().sum()

0

In [ ]:
d_ent_preprocessing['업력(연)'] = 0
d_ent_preprocessing['업력(월)'] = 0

In [ ]:
# 업력(연) 적용 함수
def enterprise_age_year(string):
    if (string == '0.0')|(string == '')|(string == '0'):
        string = '측정제한'
    else:
        today = datetime.today()
        y = today.year
        m = today.month
        d = today.day
        if int(string[4:6]) < m:
            string = y - int(string[:4])
        elif int(string[4:6]) == m:
            if int(string[6:8]) <= d:
                string = y - int(string[:4])
            elif int(string[6:8]) > d:
                string = y - int(string[:4]) - 1
        elif int(string[4:6]) > m:
            string = y - int(string[:4]) - 1
    return string

# 업력(월) 적용함수
def enterprise_age_month(string):
    if (string == '0.0')|(string == '')|(string == '0'):
        string = '측정제한'
    else:
        today = datetime.today()
        y = today.year
        m = today.month
        d = today.day
        if int(string[4:6]) < m:
            string = (y - int(string[:4])) * 12 + int(string[4:6])
        elif int(string[4:6]) == m:
            if int(string[6:8]) <= d:
                string = (y - int(string[:4])) * 12 + int(string[4:6])
            elif int(string[6:8]) > d:
                string = (y - int(string[:4]) -1) * 12 + int(string[4:6])
        elif int(string[4:6]) > m:
            string = (y - int(string[:4]) -1) * 12 + int(string[4:6])
    return string

In [ ]:
d_ent_preprocessing['업력(연)'] = d_ent_preprocessing['기업설립일자'].apply(enterprise_age_year)
d_ent_preprocessing['업력(월)'] = d_ent_preprocessing['기업설립일자'].apply(enterprise_age_month)

In [ ]:
d_ent_preprocessing.dtypes

사업자등록번호        float64
기업체명            object
기업기본주소          object
종업원수             int64
기업설립일자          object
금융감독원법인고유번호      int64
업종              object
법인등록번호         float64
기업규모            object
업력(연)           object
업력(월)           object
dtype: object

### 지역 분리 ###

In [ ]:
# 지역 분리
d_ent_preprocessing['지역(광역시도)'] = '-'
d_ent_preprocessing['지역(시군구)'] = '-'

In [ ]:
for i in range(len(d_ent_preprocessing)):
    s = d_ent_preprocessing['기업기본주소'][i].split(' ')
    d_ent_preprocessing['지역(광역시도)'][i] = s[0]
    d_ent_preprocessing['지역(시군구)'][i] = s[1]

In [ ]:
d_ent_preprocessing.head(1)

,사업자등록번호,기업체명,기업기본주소,종업원수,기업설립일자,금융감독원법인고유번호,업종,법인등록번호,기업규모,업력(연),업력(월),지역(광역시도),지역(시군구)
0,3.128135e+09,(주)다코,충청남도 천안시 청당동 419-12,0,19970611.0,434003,제조업,1.615110e+12,중소기업,24,294,충청남도,천안시


In [ ]:
d_ent_preprocessing.to_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/d_ent_final.csv', encoding='utf-8-sig', index=False)

In [ ]:
d_ent_preprocessing.shape

(2233, 13)

## 기존 기업 데이터와 새로운 기업 데이터 합치기 ##

In [ ]:
d_ent_new = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/d_ent_final.csv')
d_ent_old = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/d_ent_ver1.csv')

In [ ]:
d_ent_new.head()

,사업자등록번호,기업체명,기업기본주소,종업원수,기업설립일자,금융감독원법인고유번호,업종(대분류),법인등록번호,기업규모,업력(연),업력(월),지역(광역시도),지역(시군구)
0,3.128135e+09,(주)다코,충청남도 천안시 청당동 419-12,0,19970611,434003,제조업,1.615110e+12,중소기업,24,294,충청남도,천안시
1,2.208193e+09,주식회사포스미디어,경기도 군포시 금정동 690-4,0,20000710,430229,제조업,1.101110e+12,중소기업,21,259,경기도,군포시
2,1.058181e+09,주식회사 예전미디어,서울특별시 마포구 서교동 347-18 윤우빌딩 지하,0,19960726,405658,정보통신업,1.101110e+12,중소기업,25,307,서울특별시,마포구
3,1.108135e+09,(주)승진씨앤씨,서울특별시 중구 서소문동 75,0,19721208,218229,부동산업,1.650110e+12,중소기업,49,600,서울특별시,중구
4,2.020000e+02,글로벌렌탈 주식회사,서울특별시 강남구 삼성동 141-28 동신빌딩 4층,0,19950518,230407,"사업시설 관리, 사업 지원 및 임대 서비스업",1.101110e+12,중소기업,26,317,서울특별시,강남구


In [ ]:
d_ent_new.shape

(2233, 13)

In [ ]:
d_ent_new = d_ent_new.rename(columns={'업종':'업종(대분류)'})

In [ ]:
d_ent_old.shape

(352, 7)

In [ ]:
d_ent_old.head()

,기업체명,업력(월),기업형태,기업규모,업종(대분류),지역(광역시도),지역(시군구)
0,금융감독원,277,일반법인,판단제외,금융 및 보험업,서울특별시,영등포구
1,(주)명현유통,244,외감,중기업,도매 및 소매업,경기도,이천시
2,(주)용준,28,일반법인,소상공인,도매 및 소매업,광주광역시,서구
3,웰체인쉽핑(주),184,일반법인,중기업,운수 및 창고업,울산광역시,남구
4,(재단)전북연구원,195,일반법인,판단제외,"전문, 과학 및 기술 서비스업",전라북도,전주시


In [ ]:
d_ent_old_concat = d_ent_old[['기업체명','업력(월)','기업규모','업종(대분류)','지역(광역시도)','지역(시군구)']]
d_ent_new_concat = d_ent_new[['기업체명','업력(월)','기업규모','업종(대분류)','지역(광역시도)','지역(시군구)']]

In [ ]:
d_ent_concat = pd.concat([d_ent_old_concat, d_ent_new_concat])
d_ent_concat

,기업체명,업력(월),기업규모,업종(대분류),지역(광역시도),지역(시군구)
0,금융감독원,277,판단제외,금융 및 보험업,서울특별시,영등포구
1,(주)명현유통,244,중기업,도매 및 소매업,경기도,이천시
2,(주)용준,28,소상공인,도매 및 소매업,광주광역시,서구
3,웰체인쉽핑(주),184,중기업,운수 및 창고업,울산광역시,남구
4,(재단)전북연구원,195,판단제외,"전문, 과학 및 기술 서비스업",전라북도,전주시
...,...,...,...,...,...,...
2228,주식회사 스틸코리아,284,중소기업,제조업,서울특별시,서초구
2229,(주)인엑스하우징,252,중소기업,부동산업,서울특별시,강남구
2230,(주)우리테크놀로지,117,중소기업,제조업,경기도,군포시
2231,(주)에스앤제이케이,271,중소기업,부동산업,서울특별시,강남구


In [ ]:
d_ent_concat['기업규모'].unique()

array(['판단제외', '중기업', '소상공인', '판단불가', '소기업', '중견기업', '대기업', '중소기업', '기타'],
      dtype=object)

In [ ]:
d_ent_concat.dtypes

기업체명        object
업력(월)       object
기업규모        object
업종(대분류)     object
지역(광역시도)    object
지역(시군구)     object
dtype: object

In [ ]:
d_ent_concat['기업규모'] = d_ent_concat['기업규모'].fillna('판단불가')

In [ ]:
d_ent_concat[(d_ent_concat['기업규모'] == '-')] = '판단불가'
d_ent_concat[(d_ent_concat['기업규모'] == '보호대상중견기업')] = '중견기업'

In [ ]:
d_ent_concat.reset_index(inplace=True)

In [ ]:
d_ent_concat.drop('index',axis=1,inplace=True)

In [ ]:
d_ent_concat

,기업체명,업력(월),기업규모,업종(대분류),지역(광역시도),지역(시군구)
0,금융감독원,277,판단제외,금융 및 보험업,서울특별시,영등포구
1,(주)명현유통,244,중기업,도매 및 소매업,경기도,이천시
2,(주)용준,28,소상공인,도매 및 소매업,광주광역시,서구
3,웰체인쉽핑(주),184,중기업,운수 및 창고업,울산광역시,남구
4,(재단)전북연구원,195,판단제외,"전문, 과학 및 기술 서비스업",전라북도,전주시
...,...,...,...,...,...,...
2580,주식회사 스틸코리아,284,중소기업,제조업,서울특별시,서초구
2581,(주)인엑스하우징,252,중소기업,부동산업,서울특별시,강남구
2582,(주)우리테크놀로지,117,중소기업,제조업,경기도,군포시
2583,(주)에스앤제이케이,271,중소기업,부동산업,서울특별시,강남구


In [ ]:
d_ent_concat.to_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/기업 데이터 최종/d_ent_concat.csv', encoding='utf-8-sig', index=False)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
d_ent_train, d_ent_test = train_test_split(d_ent_concat, test_size=0.2, shuffle=True)

In [ ]:
d_ent_train.to_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/기업 데이터 최종/d_ent_train.csv', encoding='utf-8-sig', index=False)
d_ent_test.to_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/기업 데이터 최종/d_ent_test.csv', encoding='utf-8-sig', index=False)